In [180]:
import numpy as np 
import torch 
import pandas as pd
import yfinance as yf
from arch import arch_model
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.optimize import minimize

In [181]:
data = yf.download('^GSPC', start="2015-01-01", end="2025-01-01")

[*********************100%***********************]  1 of 1 completed


In [182]:
data.reset_index(inplace=True)

In [183]:
data = data.Close

In [184]:
data.reset_index(inplace=True)

In [185]:
data['log_returns'] = np.log(data['^GSPC']/data['^GSPC'].shift(-1)) #non-squared 
data['volatility'] = data['log_returns'].rolling(window=5).apply(lambda x: (np.sqrt(np.sum(x**2)))) #non-sduared volatility
data['volatility'] = data['volatility']*100
data['log_returns'] = data['log_returns']*100
data.drop(['^GSPC','index'], axis=1, inplace=True)
data.fillna(0, inplace=True)

In [186]:
train_len = int(len(data) * 0.8)
val_len = int(len(data)*0.1 + train_len)
test_len = int(len(data)-train_len-val_len)

In [187]:
train_data = data.iloc[:train_len]
val_data = data.iloc[train_len:val_len]
test_data = data.iloc[val_len:int(len(data))]

In [188]:
train_data

Ticker,log_returns,volatility
0,1.844721,0.000000
1,0.893325,0.000000
2,-1.156274,0.000000
3,-1.773017,0.000000
4,0.843932,3.064932
...,...,...
2007,1.455714,2.526099
2008,-0.585095,2.338602
2009,0.405784,2.194310
2010,1.209347,2.503352


In [189]:
garch = arch_model(train_data['log_returns'], vol='GARCH', p=1,q=1, mean='Zero')
garch_fit = garch.fit()

Iteration:      1,   Func. Count:      5,   Neg. LLF: 11170.316193628138
Iteration:      2,   Func. Count:     15,   Neg. LLF: 4643.98763551963
Iteration:      3,   Func. Count:     22,   Neg. LLF: 5106.335316298678
Iteration:      4,   Func. Count:     28,   Neg. LLF: 2635.042239531366
Iteration:      5,   Func. Count:     32,   Neg. LLF: 2635.042094583123
Iteration:      6,   Func. Count:     35,   Neg. LLF: 2635.0420945838796
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2635.042094583123
            Iterations: 6
            Function evaluations: 35
            Gradient evaluations: 6


In [190]:
garch_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                       Zero Mean - GARCH Model Results                        
==============================================================================
Dep. Variable:            log_returns   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2635.04
Distribution:                  Normal   AIC:                           5276.08
Method:            Maximum Likelihood   BIC:                           5292.90
                                        No. Observations:                 2012
Date:                Wed, Mar 19 2025   Df Residuals:                     2012
Time:                        00:43:18   Df Model:                            0
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0385  1.101e-02      3.497  4.697e-04 [1.693e-02,6.010e-02]
alpha[1]       0.1960  3.265e-02      6.001  1.957e-09     [  0.132,  0.260]
beta[1]        0.7795  3.050e-02     25.556 4.740e-144     [  0.720,  0.839]
============================================================================

Covariance estimator: robust
"""

In [191]:


def generate_ground_garch(omega, alpha, beta, n=1000):
    """
    Generates synthetic GARCH(1,1) data.
    Returns residuals (ϵ_t) and volatility (σ_t²).
    """
   

    am = arch_model(None, mean='Zero', vol='GARCH', p=1, q=1)
    params = np.array([omega, alpha, beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

class GARCH11Dataset(Dataset):
    def __init__(self, residuals, volatility):
        self.residuals = residuals
        self.volatiliy = volatility

        self.input = np.column_stack([
            np.ones_like(residuals),
            np.square(np.roll(residuals,1)),
            np.square(np.roll(volatility,1))
        ])
        self.output = np.square(np.roll(volatility,0))
        

    def __len__(self):
        return len(self.output)
    
    def __getitem__(self, index):
        input_sample = torch.tensor(self.input[index], dtype=torch.float32)
        output_sample = torch.tensor(self.output[index], dtype=torch.float32)
        return input_sample, output_sample
    
# Generate synthetic GARCH(1,1) data
residuals, volatility = generate_ground_garch(omega=0.1, alpha=0.2, beta=0.7, n=1000)

# Create the dataset
dataset = GARCH11Dataset(residuals, volatility)

# Create a DataLoader for batching
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Example: Iterate through the DataLoader
for batch_inputs, batch_targets in dataloader:
    print("Batch Inputs:", batch_inputs)
    print("Batch Targets:", batch_targets)
    
    break

Batch Inputs: tensor([[1.0000e+00, 2.9918e-02, 8.3348e-01],
        [1.0000e+00, 1.0848e+00, 1.3770e+00],
        [1.0000e+00, 2.2855e+00, 1.2809e+00],
        [1.0000e+00, 9.8939e-02, 1.4537e+00],
        [1.0000e+00, 4.5515e-03, 1.1374e+00],
        [1.0000e+00, 1.1093e-01, 8.9708e-01],
        [1.0000e+00, 2.6595e-01, 7.5014e-01],
        [1.0000e+00, 1.3815e+00, 6.7829e-01],
        [1.0000e+00, 3.8093e-01, 8.5110e-01],
        [1.0000e+00, 6.2615e-01, 7.7196e-01],
        [1.0000e+00, 5.5434e-02, 7.6560e-01],
        [1.0000e+00, 8.0816e-01, 6.4701e-01],
        [1.0000e+00, 2.9182e+00, 7.1454e-01],
        [1.0000e+00, 7.4784e-02, 1.1838e+00],
        [1.0000e+00, 4.1292e-01, 9.4362e-01],
        [1.0000e+00, 3.6793e-01, 8.4312e-01],
        [1.0000e+00, 2.2497e-01, 7.6377e-01],
        [1.0000e+00, 7.2304e-01, 6.7963e-01],
        [1.0000e+00, 7.1127e-04, 7.2035e-01],
        [1.0000e+00, 1.2706e-01, 6.0439e-01],
        [1.0000e+00, 4.0423e+00, 5.4848e-01],
        [1.0000e+00,

In [192]:
def generate_ground_gjrgarch(omega, alpha, lmbda, beta, n = 1000):
    am = arch_model(None, mean='Zero', p =1, q = 1, o =1)
    params = np.array([omega,alpha,lmbda,beta])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [193]:
def generate_ground_figarch(omega, beta, phi ,d, n = 1000):
    am = arch_model(None, mean='Zero', vol='FIGARCH')
    params= np.array([omega, beta, phi, d])
    am_data = am.simulate(params, n)

    return am_data['data'].to_numpy(), am_data['volatility'].to_numpy()

In [194]:
generate_ground_garch(0.5,0.2,0.1);

In [195]:
generate_ground_gjrgarch(0.1,0.2,0.3,0.4);

In [196]:
generate_ground_figarch(0.1,0.2,0.3,0.4);

In [197]:
class RNNGARCH (torch.nn.Module): 
    def __init__(self,  input_size=3, hidden_size=1):
        super(RNNGARCH, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.linear = nn.Linear(3, 1, bias=False)

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        sigma_t = self.linear(x)
        return sigma_t.squeeze()
    
    
    def get_garch_parameters(self) -> tuple[float, float, float]:
        """
        Returns the GARCH(1,1) parameters (ω, α, β) from the layer's weights.
        """
        weights = self.linear.weight.data.squeeze().tolist()
        omega, alpha, beta = weights
        return omega, alpha, beta

        


In [198]:
class GARCHNegativeLogLikelihood(nn.Module):
   
    def __init__(self):
        super().__init__()

    def forward(self, residuals: torch.Tensor, estimated_volatility: torch.Tensor) -> torch.Tensor:
        
        # Ensure estimated_volatility is positive to avoid numerical issues
        

        # Compute the two terms of the negative log-likelihood
        term1 = 0.5 * torch.log(estimated_volatility)  # log(σ_t²) / 2
        term2 = (residuals ** 2) / (2 * estimated_volatility)  # ϵ_t² / (2 * σ_t²)

        # Sum the terms and average over the batch
        loss = torch.mean((term1 + term2))
        return loss

In [199]:
# Create the GARCH(1,1) RNN layer
garch_layer = RNNGARCH()

# Create the loss function
loss_fn = nn.MSELoss()

# Create an optimizer
optimizer = torch.optim.Adam(garch_layer.parameters(), lr=0.0001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_inputs, batch_targets in dataloader:
        # Forward pass
        estimated_volatility = garch_layer(batch_inputs)
        residuals = torch.sqrt(batch_inputs[:,-1])
        # Compute loss
        residuals = batch_targets  # Assuming batch_targets are σ²_t, residuals are ϵ_t = sqrt(σ²_t)
        loss = loss_fn(residuals, estimated_volatility)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Print average loss for the epoch
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss / len(dataloader)}")

# Get the trained GARCH(1,1) parameters
omega, alpha, beta = garch_layer.get_garch_parameters()
print(f"Trained GARCH(1,1) Parameters: ω={omega}, α={alpha}, β={beta}")

Epoch 100, Average Loss: 0.18196030044055078
Epoch 200, Average Loss: 0.01849713049159618
Epoch 300, Average Loss: 0.008853432183968835
Epoch 400, Average Loss: 0.00331975589870126
Epoch 500, Average Loss: 0.0008417066076162882
Epoch 600, Average Loss: 0.00046252701681481767
Epoch 700, Average Loss: 0.0004614262414905923
Epoch 800, Average Loss: 0.00046155216714893754
Epoch 900, Average Loss: 0.00046154858785119046
Epoch 1000, Average Loss: 0.0004615058415113893
Trained GARCH(1,1) Parameters: ω=0.10086606442928314, α=0.19965048134326935, β=0.6997845768928528


In [86]:
#NUMERICAL UNDERVALUES 